<a href="https://www.kaggle.com/code/averma111/pytroch-llm-science-exam?scriptVersionId=138015226" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [96]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
from transformers import AutoTokenizer , AutoModel
import torch
from IPython.display import IFrame

import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
 

/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv


In [97]:
class DataAcquisition:
    
    def __init__(self):
        pass
    
    def prepare_data(self):
        path = '/kaggle/input/kaggle-llm-science-exam/train.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    def prepare_data_test(self):
        path = '/kaggle/input/kaggle-llm-science-exam/test.csv'
        df = pd.read_csv(
            path, 
            sep=',', 
            low_memory=False
        )
        
        X = df.copy()
        return X
    
    
    def new_features(self,df):
        pass
    
    
    
    def preprocessing(self,df):
        for idx in tqdm(range(df.shape[0]) , total = df.shape[0]):
            df["answer"][idx] = df[df["answer"][idx]][idx]
        return df
        
    def preprocessing_test(self,df):
        for idx in tqdm(range(df.shape[0]) , total = df.shape[0]):
            df["answer"][idx] = df[df["answer"][idx]][idx]
        return df
    
        
acq = DataAcquisition()

train_df = acq.prepare_data()
test_df = acq.prepare_data_test()

In [98]:
train_df.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [99]:
train_df=acq.preprocessing(train_df)
train_df.drop(["id" , "A" , "B" , "C" , "D" , "E"] , axis = 1 , inplace = True)
train_df.head()

  0%|          | 0/200 [00:00<?, ?it/s]

,prompt,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the discrepancy ...
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...


In [100]:
models_dict= {
    'rb':'roberta-base',
    'rbl':'roberta-large',
    'abv2':'albert-base-v2',
    'msd':'microsoft/deberta-base',
    'ggl':'google/electra-medium-discriminator',
    'fb':'facebook/bart-base'
}

In [101]:
class LLMNNBase(torch.nn.Module):
    
    def __init__(self,model_type,n_neurons):
        super(LLMNNBase, self).__init__()
        self.model_type = model_type
        self.model = AutoModel.from_pretrained(self.model_type)
        self.n_neurons = n_neurons
        self.linear_1 = torch.nn.Linear(self.n_neurons , self.n_neurons)
    
    def forward(self, inputs):
        inputs = self.model(inputs)[0]
        inputs = torch.mean(inputs, axis=1)
        output = self.linear_1(inputs)

        return output

In [102]:
def tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    for idx in tqdm(range(train_df.shape[0])):
        train_df["prompt"][idx] = tokenizer(train_df["prompt"][idx] , 
                                       return_tensors = "pt")["input_ids"]
    
    with torch.no_grad():
        train_df["answer"][idx] = model(
                tokenizer(train_df["answer"][idx] , 
                return_tensors = "pt")["input_ids"])[0][0][0]
    torch.cuda.empty_cache()
    return train_df

In [103]:
for model_key, model_value in models_dict.items():
    train_df=tokenizer(model_value)
    model = LLMNNBase(model_value)
    optim = torch.optim.Adam(model.parameters())
    criterion = torch.nn.MSELoss()
    losses = []
    
    for x , y in tqdm(zip(train_df["prompt"] , train_df["answer"]) , total = train_df.shape[0]):
        x = torch.tensor(x , dtype = torch.long).to("cuda")
        y = torch.tensor(y , dtype = torch.float32).to("cuda")
        x = x.reshape(shape = (1 , x.shape[0]))
        if x.shape[1] > 512: x = x[: , :512]
        pred = model(x)[0]
        loss_fun = criterion(pred , y)
        losses.append(loss_fun)
        torch.cuda.empty_cache()
        optim.step

  0%|          | 0/200 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 for model_key, model_value in models_dict.items():                                          │
│ ❱  2 │   train_df=tokenizer(model_value)                                                         │
│    3 │   model = LLMNNBase(model_value)                                                          │
│    4 │   optim = torch.optim.Adam(model.parameters())                                            │
│    5 │   criterion = torch.nn.MSELoss()                                                          │
│                                                                                                  │
│ in tokenizer:8                                                                                   │
│                                                                                                  │
│    5 │   │   │   │   │   │   │   │   │      return_tensors = "pt")["input_ids"]                  │
│    6 │                                                                                           │
│    7 │   with torch.no_grad():                                                                   │
│ ❱  8 │   │   train_df["answer"][idx] = model(                                                    │
│    9 │   │   │   │   tokenizer(train_df["answer"][idx] ,                                         │
│   10 │   │   │   │   return_tensors = "pt")["input_ids"])[0][0][0]                               │
│   11 │   torch.cuda.empty_cache()                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ in forward:13                                                                                    │
│                                                                                                  │
│   10 │   def forward(self, inputs):                                                              │
│   11 │   │   inputs = self.model(inputs)[0]                                                      │
│   12 │   │   inputs = torch.mean(inputs, axis=1)                                                 │
│ ❱ 13 │   │   output = self.linear_1(inputs)                                                      │
│   14 │   │                                                                                       │
│   15 │   │   return output                                                                       │
│   16                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/mo